In [13]:
import pandas as pd
import collections
import etl_idesp
pd.set_option('display.max_columns', None)

In [ ]:
# TODO - IR NA TABELA IDESP POR ESCOLA E VER SE ESTA TUDO LA

In [ ]:
# Preciso apontar as escolas que ESTAO na tabela FATO.
# Mas NAO estao nas dimensao destino (e.g. dependencias)

In [2]:
df_idesp = etl_idesp.etl()

In [42]:
df_idesp.id_escola = df_idesp.id_escola.astype(int)

In [3]:
df_idesp.head(3)

,id_gestao,id_escola,ano_gestao,nota_ano,variacao,anterior_nulo,atual_nulo
0,esc-10005-ano-2014,10005,2014,3.48,10.828025,0,1
1,esc-10005-ano-2015,10005,2015,3.78,8.620690,0,1
2,esc-10005-ano-2016,10005,2016,3.26,-13.756614,0,1


In [22]:
escolas_fato = set(df_idesp.id_escola)

In [5]:
esc = {}
esc['id_escola'] = []
esc['nota'] = []
for nome, dados in df_idesp.groupby('id_escola'):
    esc['id_escola'].append(nome)
    esc['nota'].append(dados.nota_ano.mean())
esc = pd.DataFrame(esc)

# Dimensao escola (associada a uma dada gestao)
- Escola fisica.
- Classes.
- Idesp: Nome diretoria.
- Endereco.
- Dependencias fisicas

In [11]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\instalacao_fisicas\\06_Escolas_Dependencias.csv'
dep = pd.read_csv(arquivo, sep=';')

In [44]:
dep.CODESC = dep.CODESC.astype(int)
escolas_dep = set(dep.CODESC)

In [83]:
dep = dep.rename({'CODESC': 'id_escola'}, axis=1)

In [14]:
dep.head(3)

,NOMEDEP,DE,MUN,DISTR,CODESC,NOMESC,TIPOESC,TIPOESC_DESC,SITUACAO,SALAS_AULA,SALAS_ED_INF,SALAS_ED_ESP,SALAS_ED_ART,SALA_RECURSO,TOT_SALAS_AULA,AUDITORIO,ANFITEATRO,TEATRO,CANTINA,COPA,COZINHA,REFEITORIO,DEPOSITO_ALIMENTOS,DESPENSA,TOT_DESPENSA,SALA_LEITURA,BIBLIOTECA,TOT_SALA_LEITURA,QUADRA_COBERTA,QUADRA_DESCOBERTA,GINASIO,TOT_QUADRA,QUADRA_AREIA,QUADRA_GRAMA,CAMPO_FUTEBOL,GABINETE_DENTARIO,CONSULTORIO_MEDICO,ENFERMARIA,AMBULATORIO,ALMOXARIFADO,ARQUIVO,REPROGRAFIA,SALA_GREMIO,DIRETORIA,VICEDIRETORIA,SALA_PROF,SECRETARIA,SALA_ORIENT_ED,SALA_COORD_PEDAG,PATIO_COBERTO,PATIO_DESCOBERTO,ZELADORIA,VESTIARIO_FEM,VESTIARIO_MASC,TOT_VESTIARIO,VIDEOTECA,SALA_TV,LAB_INFO,LAB_CIENCIAS,LAB_FISICA,LAB_QUIMICA,LAB_BIOLOGIA,LAB_CIENCIA_FISICA_BIOLOGICA,TOT_LAB_CIENCIA,LAB_LINGUAS,LAB_MULTIUSO,OFICINA,PLAYGROUND,DORMITORIO,BERCARIO,SANITARIO_ADEQ_PRE,SANITARIO_ADEQ_PRE_FEM,SANITARIO_ADEQ_PRE_MASC,SANITARIO_ADEQ_DEF,SANITARIO_ADEQ_DEF_FEM,SANITARIO_ADEQ_DEF_MASC,SANITARIO_AL_MASC,SANITARIO_AL_FEM,TOT_SANITARIO_AL,SANITARIO_FUNC_FEM,SANITARIO_FUNC_MASC,TOT_SANITARIO_FUNC,DEPEND_ADEQ_DEF,SALA_ED_FISICA,PISCINA,PORTARIA,SALA_PROG_ESC_FAMILIA,BRINQUEDOTECA,FRALDARIO,LACTARIO,LAVANDERIA,SOLARIO,SALA_ESPERA,SALA_INSPETOR,SALA_REUNIAO,TESOURARIA,SALA_REFORCO,SALA_DIRETOR_TECNICO,GARAGEM_ONIBUS,SALA_FISIOTERAPIA,SALA_PSICOLOGIA,SALA_FONOAUDIOLOGIA,SALA_EVENTOS,SALA_ASSIST_SOCIAL,SALA_TERAPIA_EDUC,ABATEDOURO,ALOJAMENTO_FEM,ALOJAMENTO_MASC,TOT_ALOJAMENTO,AREA_SERVICO,BAZAR,CASA_MAQUINA,CASA_FUNC,CHURRASQUEIRA,DEPOSITOS_CEREAIS,ELEVADOR,ESTACIONAMENTO,ESTUFA,GALPAO_AVES_CORTE,GALPAO_AVES_POSTURA,GALPAO_BOVINOS_LEITE,GALPAO_CUNICULTURA,GALPAO_MAQ_AGRICOLA,GALPAO_OVINOS_CAPRINOS,GALPAO_SUINO,GRAFICA,HORTA,LAB_DIDATICA,LAB_JUNIOR,LAB_ENFERMAGEM,LAB_ESTETICA,LAB_PSICOPEDAGOGIA,LAB_TURISMO,LAVATORIO,MANGUEIRA,MINHOCARIO,PACKING_HOUSE,POMAR,PSICULTURA,RECEPCAO,SALA_ATENDIMENTO,SALA_ATEND_PSICOLOGICO,SALA_AUX_COORDENACAO,SALA_DADOS,SALA_DEP_PESSOAL,SALA_ED_RELIGIOSA,SALA_ENERGIA_ELETRICA,SALA_ENTRETENIMENTO,SALA_ESTAGIO,SALA_GINASTICA,SALA_INSUMO_AGRICOLA,SALA_INSUMO_VETERINARIO,SALA_MARKETING,SALA_MATRICULA,SALA_MUSICA,SALA_POS_GRADUACAO,SALA_ORDENHA,SALA_PROC_PROD_AGROPECUARIOS,SALA_SEGURANCA,SALA_TELEFONIA,SALA_FINANCEIRO,SALA_PASTORAL,SALA_RESERV_AGUA,SERVIDOR,SILO,VARANDA,VIVEIRO,SALA_REORG_NEURO,SALA_TERAPIA_OCUP,SALA_SERIGRAFIA,SALA_MARCENARIA,QUIOSQUE
0,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,985429,CEL JTO A EE FLEURIDES CAVALINI MENECHINO PROFA,6,CEL,Ativa,5,0,0,0,0,5,0,0,0,0,0,1,1,0,0,0,1,0,1,1,1,0,2,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,1,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31045,DURVALINO GRION PROF,8,EE,Ativa,9,0,0,0,0,9,0,0,0,0,1,1,0,0,1,1,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,0,1,2,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,2,1,1,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31112,FLEURIDES CAVALLINI MENECHINO PROFA,8,EE,Ativa,12,0,0,0,1,13,0,0,0,0,1,1,1,0,1,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,5,1,0,0,1,1,1,1,0,1,2,1,1,1,1,2,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,2,3,5,1,1,2,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
dep = dep.drop([
    'NOMEDEP', # todas sao 'ESTADUAL - SE'
    'SITUACAO', # todos ativos
    'DE' # diretoria vai vir da tabela idesp
],axis=1)

# DE A diretoria vai vir da tabela idesp


In [85]:
idesp = pd.merge(df_idesp, dep, on='id_escola',how='left')

In [89]:
idesp.isnull().sum()

id_gestao                          0
id_escola                          0
ano_gestao                         0
nota_ano                        1939
variacao                        1939
anterior_nulo                      0
atual_nulo                         0
DE                              1745
MUN                             1745
DISTR                           1745
NOMESC                          1745
TIPOESC                         1745
TIPOESC_DESC                    1745
SITUACAO                        1745
SALAS_AULA                      1745
SALAS_ED_INF                    1745
SALAS_ED_ESP                    1745
SALAS_ED_ART                    1745
SALA_RECURSO                    1745
TOT_SALAS_AULA                  1745
AUDITORIO                       1745
ANFITEATRO                      1745
TEATRO                          1745
CANTINA                         1745
COPA                            1745
COZINHA                         1745
REFEITORIO                      1745
D

In [79]:
dep.columns

Index(['DE', 'MUN', 'DISTR', 'CODESC', 'NOMESC', 'TIPOESC', 'TIPOESC_DESC',
       'SITUACAO', 'SALAS_AULA', 'SALAS_ED_INF',
       ...
       'SALA_RESERV_AGUA', 'SERVIDOR', 'SILO', 'VARANDA', 'VIVEIRO',
       'SALA_REORG_NEURO', 'SALA_TERAPIA_OCUP', 'SALA_SERIGRAFIA',
       'SALA_MARCENARIA', 'QUIOSQUE'],
      dtype='object', length=171)

In [82]:
dep.SERVIDOR.describe()

count    5608.000000
mean        0.000357
std         0.018883
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: SERVIDOR, dtype: float64

In [78]:
dep[dep.SALAS_AULA == 1]

,DE,MUN,DISTR,CODESC,NOMESC,TIPOESC,TIPOESC_DESC,SITUACAO,SALAS_AULA,SALAS_ED_INF,SALAS_ED_ESP,SALAS_ED_ART,SALA_RECURSO,TOT_SALAS_AULA,AUDITORIO,ANFITEATRO,TEATRO,CANTINA,COPA,COZINHA,REFEITORIO,DEPOSITO_ALIMENTOS,DESPENSA,TOT_DESPENSA,SALA_LEITURA,BIBLIOTECA,TOT_SALA_LEITURA,QUADRA_COBERTA,QUADRA_DESCOBERTA,GINASIO,TOT_QUADRA,QUADRA_AREIA,QUADRA_GRAMA,CAMPO_FUTEBOL,GABINETE_DENTARIO,CONSULTORIO_MEDICO,ENFERMARIA,AMBULATORIO,ALMOXARIFADO,ARQUIVO,REPROGRAFIA,SALA_GREMIO,DIRETORIA,VICEDIRETORIA,SALA_PROF,SECRETARIA,SALA_ORIENT_ED,SALA_COORD_PEDAG,PATIO_COBERTO,PATIO_DESCOBERTO,ZELADORIA,VESTIARIO_FEM,VESTIARIO_MASC,TOT_VESTIARIO,VIDEOTECA,SALA_TV,LAB_INFO,LAB_CIENCIAS,LAB_FISICA,LAB_QUIMICA,LAB_BIOLOGIA,LAB_CIENCIA_FISICA_BIOLOGICA,TOT_LAB_CIENCIA,LAB_LINGUAS,LAB_MULTIUSO,OFICINA,PLAYGROUND,DORMITORIO,BERCARIO,SANITARIO_ADEQ_PRE,SANITARIO_ADEQ_PRE_FEM,SANITARIO_ADEQ_PRE_MASC,SANITARIO_ADEQ_DEF,SANITARIO_ADEQ_DEF_FEM,SANITARIO_ADEQ_DEF_MASC,SANITARIO_AL_MASC,SANITARIO_AL_FEM,TOT_SANITARIO_AL,SANITARIO_FUNC_FEM,SANITARIO_FUNC_MASC,TOT_SANITARIO_FUNC,DEPEND_ADEQ_DEF,SALA_ED_FISICA,PISCINA,PORTARIA,SALA_PROG_ESC_FAMILIA,BRINQUEDOTECA,FRALDARIO,LACTARIO,LAVANDERIA,SOLARIO,SALA_ESPERA,SALA_INSPETOR,SALA_REUNIAO,TESOURARIA,SALA_REFORCO,SALA_DIRETOR_TECNICO,GARAGEM_ONIBUS,SALA_FISIOTERAPIA,SALA_PSICOLOGIA,SALA_FONOAUDIOLOGIA,SALA_EVENTOS,SALA_ASSIST_SOCIAL,SALA_TERAPIA_EDUC,ABATEDOURO,ALOJAMENTO_FEM,ALOJAMENTO_MASC,TOT_ALOJAMENTO,AREA_SERVICO,BAZAR,CASA_MAQUINA,CASA_FUNC,CHURRASQUEIRA,DEPOSITOS_CEREAIS,ELEVADOR,ESTACIONAMENTO,ESTUFA,GALPAO_AVES_CORTE,GALPAO_AVES_POSTURA,GALPAO_BOVINOS_LEITE,GALPAO_CUNICULTURA,GALPAO_MAQ_AGRICOLA,GALPAO_OVINOS_CAPRINOS,GALPAO_SUINO,GRAFICA,HORTA,LAB_DIDATICA,LAB_JUNIOR,LAB_ENFERMAGEM,LAB_ESTETICA,LAB_PSICOPEDAGOGIA,LAB_TURISMO,LAVATORIO,MANGUEIRA,MINHOCARIO,PACKING_HOUSE,POMAR,PSICULTURA,RECEPCAO,SALA_ATENDIMENTO,SALA_ATEND_PSICOLOGICO,SALA_AUX_COORDENACAO,SALA_DADOS,SALA_DEP_PESSOAL,SALA_ED_RELIGIOSA,SALA_ENERGIA_ELETRICA,SALA_ENTRETENIMENTO,SALA_ESTAGIO,SALA_GINASTICA,SALA_INSUMO_AGRICOLA,SALA_INSUMO_VETERINARIO,SALA_MARKETING,SALA_MATRICULA,SALA_MUSICA,SALA_POS_GRADUACAO,SALA_ORDENHA,SALA_PROC_PROD_AGROPECUARIOS,SALA_SEGURANCA,SALA_TELEFONIA,SALA_FINANCEIRO,SALA_PASTORAL,SALA_RESERV_AGUA,SERVIDOR,SILO,VARANDA,VIVEIRO,SALA_REORG_NEURO,SALA_TERAPIA_OCUP,SALA_SERIGRAFIA,SALA_MARCENARIA,QUIOSQUE
185,APIAI,IPORANGA,IPORANGA,6283,BAIRRO BOMBAS DE CIMA,31,QUILOMBO,Ativa,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
186,APIAI,IPORANGA,IPORANGA,5693,BAIRRO DE BOMBAS,31,QUILOMBO,Ativa,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
189,APIAI,ITAOCA,ITAOCA,317724,BAIRRO CANGUME,31,QUILOMBO,Ativa,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
418,BARRETOS,GUAIRA,GUAIRA,458193,CEL JTO A EE ENOCH GARCIA LEAL,6,CEL,Ativa,1,0,0,0,0,1,0,0,0,1,0,1,1,1,1,2,1,0,1,1,1,0,2,0,0,0,0,0,0,0,1,1,0,0,1,1,1,1,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,2,1,3,1,1,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
721,CAIEIRAS,FRANCO DA ROCHA,FRANCO DA ROCHA,458107,HC

# Dimensao Servidores (associados a uma dada gestao)
- Servidores da escola.
- Servidores ativos: Vou colocar os servidores na escola correspondente.
- Atribuicao digital: Disciplinas sem docente.
- Ausencias por servidor: Numero total de ausencias da escola.
- Carga horaria.
- Cluster.

# CLUSTER
- CD_CLUSTERS e o id
- CD_DIRETORIA ha 91 diretorias
- Numero do cluster: 9
- NR_CICLO: 3
- CD_GRUPO_CLUSTER

In [6]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\cluster_de_escolas_similares\\12_MMR_Clusters.csv'
cluster = pd.read_csv(arquivo, sep=';')
cluster.CD_ESCOLA = cluster.CD_ESCOLA.astype(int)
cluster = cluster.rename({'CD_ESCOLA': 'id_escola'}, axis=1)

In [7]:
clu = {}
clu['id_escola'] = []
clu['nr_cluster'] = []
for nome, dados in cluster.groupby('id_escola'):
    clu['id_escola'].append(nome)
    clu['nr_cluster'].append(dados.NR_CLUSTER.max())  


In [8]:
clu = pd.DataFrame(clu)

In [9]:
esc_cl = pd.merge(esc, clu, on='id_escola',how='left')

In [10]:
esc_cl.loc[:,['nota','nr_cluster']].corr()

,nota,nr_cluster
nota,1.000000,-0.220918
nr_cluster,-0.220918,1.000000


In [11]:
esc_cl.nr_cluster.describe()

count    2134.000000
mean        4.126054
std         2.022273
min         1.000000
25%         2.000000
50%         4.000000
75%         6.000000
max         9.000000
Name: nr_cluster, dtype: float64

In [30]:
esc_cl[esc_cl.nr_cluster.isin([1,2,3,4])].nota.mean()

4.40099133369823

In [39]:
esc_cl[esc_cl.nr_cluster.isin([1,2,3,4])].nota.count()

1218

In [31]:
esc_cl[esc_cl.nr_cluster.isin([5,9])].nota.mean()

4.121484203296703

In [40]:
esc_cl[esc_cl.nr_cluster.isin([5,9])].nota.count()

364

In [32]:
esc_cl[esc_cl.nr_cluster.isin([6,7,8])].nota.mean()

3.5821358192431565

In [41]:
esc_cl[esc_cl.nr_cluster.isin([6,7,8])].nota.count()

552

In [33]:
esc_cl[esc_cl.nr_cluster == 0].nota.mean()

2.776062744507975

In [38]:
esc_cl[esc_cl.nr_cluster == 0].count()

id_escola     3323
nota          3323
nr_cluster    3323
dtype: int64

# NUMERO DE CLASSES

In [30]:
arquivo = 'dados\matriculas\/10_Escolas_Classes_Qtde_Alunos.csv'
classes = pd.read_csv(arquivo, sep=';')

In [32]:
classes.COD_ESC = classes.COD_ESC.astype(int)

In [34]:
classes = classes.rename({'COD_ESC': 'id_escola'}, axis=1)

In [ ]:
result = pd.merge(esc_cl, )

In [36]:
classes.head(10)

,ANO,NOMEDEP,DE,MUN,DISTR,id_escola,NOMESC,TIPOESC,TIPOESC_DESC,SITUACAO,CLASSE,TIPOCLASSE,TIPOCLASSE_DESC,GRAU,TipoEnsino,SERIE,QTDE_ALUNOS,CODVINC
0,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,30806,HELEN KELLER,8,EE,Ativa,182570713,0,CLASSE COMUM,1,ENSINO FUNDAMENTAL DE 8 ANOS,8,21,NaN
1,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,30806,HELEN KELLER,8,EE,Ativa,182570762,0,CLASSE COMUM,2,ENSINO MEDIO,2,37,NaN
2,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,30806,HELEN KELLER,8,EE,Ativa,186061859,1,CLASSE MULTISSERIADA,5,EJA ENSINO MÉDIO,10,25,NaN
3,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31045,DURVALINO GRION PROF,8,EE,Ativa,182570440,0,CLASSE COMUM,2,ENSINO MEDIO,2,26,NaN
4,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31045,DURVALINO GRION PROF,8,EE,Ativa,182570465,0,CLASSE COMUM,2,ENSINO MEDIO,3,33,NaN
5,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31045,DURVALINO GRION PROF,8,EE,Ativa,182570523,0,CLASSE COMUM,14,ENSINO FUNDAMENTAL DE 9 ANOS,7,30,NaN
6,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31112,FLEURIDES CAVALLINI MENECHINO PROFA,8,EE,Ativa,182570127,0,CLASSE COMUM,1,ENSINO FUNDAMENTAL DE 8 ANOS,7,28,NaN
7,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31112,FLEURIDES CAVALLINI MENECHINO PROFA,8,EE,Ativa,182570184,0,CLASSE COMUM,1,ENSINO FUNDAMENTAL DE 8 ANOS,8,35,NaN
8,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31112,FLEURIDES CAVALLINI MENECHINO PROFA,8,EE,Ativa,182570192,0,CLASSE COMUM,2,ENSINO MEDIO,1,35,NaN
9,2014,ESTADUAL - SE,ADAMANTINA,ADAMANTINA,ADAMANTINA,31112,FLEURIDES CAVALLINI MENECHINO PROFA,8,EE,Ativa,182570218,0,CLASSE COMUM,2,ENSINO MEDIO,1,32,NaN
